In [28]:
import tensorflow as tf
import keras
from keras.layers import Dense, Dropout, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
import pandas as pd

In [29]:
path = "gesture_data"

data = []
labels = []

for file_name in os.listdir(path):
    if file_name .endswith(".csv"):
        label = file_name.split("_")[0]
        file_path = os.path.join(path, file_name)

        df = pd.read_csv(file_path)
        data.append(df.values)
        labels.extend([label] * len(df))

In [30]:
X = np.vstack(data)
y = np.array(label)

encoded_label = ["like", "pause", "resume", "faster", "next", "previous"]
encoder = LabelEncoder()
encoder.fit(encoded_label)
y_encoded = encoder.fit_transform(y)

/home/gins/anaconda3/envs/tf/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
model = keras.Sequential([
    Input(shape=(X.shape[1],)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

In [32]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_entropy',
    metrics=['accuracy']
)

In [33]:
history = model.fit(X, y_encoded, epochs=40, batch_size=32, validation_split=0.2)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 303
'y' sizes: 5
